In [108]:
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta 
import scrap_titres as st



import urllib.request
from urllib.error import HTTPError


In [2]:
from inspect import getmembers, isfunction

#from somemodule import foo
print(getmembers(st, isfunction))

[('recup_listes_charlie', <function recup_listes_charlie at 0x000001D6D851ED38>), ('scrap_charlie', <function scrap_charlie at 0x000001D6D851EDC8>), ('scrap_closer', <function scrap_closer at 0x000001D6D7CF3288>), ('scrap_figaro', <function scrap_figaro at 0x000001D6D80255E8>), ('scrap_gorafi', <function scrap_gorafi at 0x000001D6D8025A68>), ('scrap_le_point', <function scrap_le_point at 0x000001D6D8536D38>), ('scrap_libe', <function scrap_libe at 0x000001D6D851EB88>), ('scrap_monde', <function scrap_monde at 0x000001D6CF48ADC8>), ('scrap_nord_presse', <function scrap_nord_presse at 0x000001D6D8025EE8>), ('scrap_public', <function scrap_public at 0x000001D6D851EC18>), ('scrap_sience_avenir', <function scrap_sience_avenir at 0x000001D6D851ECA8>), ('urlopen', <function urlopen at 0x000001D6CE461CA8>)]


In [48]:
#recupération dernier jour de scrap
f = open("dernier_jour.txt", encoding="utf-8")
day = [date.strip() for date in f][-1]
day = (pd.to_datetime(day , format="%d-%m-%Y").date())
f.close()

In [49]:
day

datetime.date(2021, 2, 1)

In [7]:
#hier
hier = date.today() - timedelta(days=1)

In [8]:
hier

datetime.date(2021, 2, 4)

In [33]:
l1 = [1,2,3,4]
l2 = []
l3 = [5,6]
l1+l2+l3

[1, 2, 3, 4, 5, 6]

In [46]:
l1=l1+l3

In [14]:
day

datetime.date(2021, 2, 4)

In [3]:
def crea_listes(x,y,taille):
    return [x]*taille,[y]*taille

In [110]:
########################
f = open("dernier_jour.txt", encoding="utf-8")
day = [date.strip() for date in f][-1]
day = (pd.to_datetime(day , format="%d-%m-%Y").date())
f.close()

#############################
#scrap chaque media tq day =! hier
liste_ch = () #pour récupération page charlie non opti
liste_titre_charlie = []
liste_th_charlie = []
liste_date_charlie = []

liste_titres = []
liste_journaux = []
liste_dates = []

noms_journaux = ['Libération','Le Monde','Nord Presse','Science et Avenir','Closer','Gorafi','La Point','Public','Figaro']
noms_function = ['scrap_libe','scrap_monde','scrap_nord_presse','scrap_sience_avenir','scrap_closer','scrap_gorafi','scrap_le_point','scrap_public','scrap_figaro']

def insert_nom_et_dates(nomJournal, day, titres_scrap):
    global liste_titres
    global liste_journaux
    global liste_dates
    
    liste_titres = liste_titres + titres_scrap
    li = crea_listes(nomJournal,day,len(titres_scrap))
    liste_journaux = liste_journaux + li[0]
    liste_dates = liste_dates + li[1]


while day!= hier:
    day = day + timedelta(days=1)
    #récupération titres charlie
    if not liste_ch:
        liste_ch = st.recup_listes_charlie(day)
    
    #pour récup theme et model en fonction => à voir en plus
    res=st.scrap_charlie(day,liste_ch) #retourne tuple titre,theme du jour
    liste_titre_charlie = liste_titre_charlie + res[0]
    liste_th_charlie = liste_th_charlie + res[1]
    liste_date_charlie = liste_date_charlie + [day]*len(res[0])
    
    #charlie pour insertion BD 
    liste_titres = liste_titres + res[0]
    li = crea_listes('Charlie Hebdo',day,len(res[0]))
    liste_journaux = liste_journaux + li[0]
    liste_dates = liste_dates + li[1]
    

    for nom_j,nom_fun in zip (noms_journaux,noms_function):
        l = []   
        try:
            l = getattr(st, nom_fun)(day)
            insert_nom_et_dates(nom_j, day, l)
            print(day, nom_j, len(l))
        except Exception as e:
            if type(e).__name__ == "HTTPError":
                print(type(e).__name__ , e.code , nom_j , len(l) ,e.url, e.name)
            else:
                print(type(e).__name__)
                
    #récupération autres journaux
#     l = st.scrap_sience_avenir(day)
#     insert_nom_et_dates('Science et Avenir', day, l)
        
#     l = st.scrap_closer(day)
#     insert_nom_et_dates('Closer', day, l)
    
#     l = st.scrap_figaro(day)
#     insert_nom_et_dates('Figaro', day, l)
    
#     l = st.scrap_gorafi(day)
#     insert_nom_et_dates('Gorafi', day, l)
    
#     l = st.scrap_le_point(day)
#     insert_nom_et_dates('Le Point', day, l)
    
#     l = st.scrap_libe(day)
#     insert_nom_et_dates('Libération', day, l)
    
#     l = st.scrap_monde(day)
#     insert_nom_et_dates('Le Monde', day, l)
    
#     try:
#         l = st.scrap_nord_presse(day)
#         insert_nom_et_dates('Nord Presse', day, l)
#     except Exception as ex:
#             print (type(ex).__name__)
#             if type(ex).__name__ == "HTTPError":
#                 print(ex.code)
#                 print(dir(ex))
#                 print('msg',ex.msg,'\n','headers',ex.headers,'\n','args',ex.args,'\n','geturl',ex.geturl,'\n')
                
# #     except urllib.error.HTTPError as err:
# #         print(err.code)     
    
#     l = st.scrap_public(day)
#     insert_nom_et_dates('Public', day, l)
    

https://www.liberation.fr/archives/2021/02/02
2021-02-02 Libération 51
2021-02-02 Le Monde 111
HTTPError 404 Nord Presse 0 https://nordpresse.be/2021/02/02/ <urllib response>
2021-02-02 Science et Avenir 25
2021-02-02 Closer 3
2021-02-02 Gorafi 3
2021-02-02 La Point 108
2021-02-02 Public 30
2021-02-02 Figaro 235
https://www.liberation.fr/archives/2021/02/03
2021-02-03 Libération 59
2021-02-03 Le Monde 121
HTTPError 404 Nord Presse 0 https://nordpresse.be/2021/02/03/ <urllib response>
2021-02-03 Science et Avenir 33
2021-02-03 Closer 44
2021-02-03 Gorafi 2
2021-02-03 La Point 108
2021-02-03 Public 34
2021-02-03 Figaro 268
https://www.liberation.fr/archives/2021/02/04
2021-02-04 Libération 66
2021-02-04 Le Monde 110
2021-02-04 Nord Presse 1
2021-02-04 Science et Avenir 34
2021-02-04 Closer 5
2021-02-04 Gorafi 3
2021-02-04 La Point 113
2021-02-04 Public 35
2021-02-04 Figaro 243


In [ ]:
['_HTTPError__super_init', '__cause__', '__class__', '__context__', '__delattr__', '__dict__', '__dir__', '__doc__', 
 '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', 
 '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__',
 '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__suppress_context__', 
 '__traceback__', '__weakref__', '_closer', 'args', 'characters_written', 'close', 'code', 'delete', 'errno', 'file', 'filename',
 'filename2', 'fp', 'getcode', 'geturl', 'hdrs', 'headers', 'info', 'msg', 'name', 'reason', 'strerror', 'url', 'winerror', 'with_traceback']

In [68]:
getattr(st, 'scrap_libe')(day)

https://www.liberation.fr/archives/2021/02/04


['Syndicat Solidaires : «Ce qu’on peut gagner, c’est un déconfinement des colères»',
 '«La Sidération» : Laurence Benaïm et le legs maternel',
 'De Sundance à Clermont, pas de festoche sans bamboche ?',
 'Les éoliennes face aux vents du passé',
 'Féminicides : «Avec une politique publique un peu volontariste et une forte mobilisation, on fait bouger les lignes»',
 'A Boulogne-Billancourt, la construction d’un palais des sports de 5 000 places fait polémique',
 '«Du jour au lendemain, notre discipline a été mise dehors»',
 'Asselineau en garde à vue, la loi bioéthique votée au Sénat sans la PMA pour toutes, plan de lutte contre le cancer... l’actu de jeudi matin',
 'En direct - Covid-19 : Confinement : «la situation ne le justifie pas», estime Jean Castex',
 'Topshop rachetée par Asos : merci patron !',
 'Rejet de la PMA pour toutes au Sénat : «La droite a un problème sérieux avec le XXIe siècle»',
 'Avec des éducateurs de la protection judiciaire de la jeunesse : «Notre rôle, c’est de 

In [90]:

liste_titres = []
liste_journaux = []
liste_dates = []


noms_journaux = ['Libération','Le Monde']#,'Nord Presse','Science et Avenir','Closer','Gorafi','La Point','Public','Figaro']
noms_function = ['scrap_libe','scrap_monde']#,'scrap_nord_presse','scrap_sience_avenir','scrap_closer','scrap_gorafi','scrap_le_point','scrap_public','scrap_figaro']
    
for nom_j,nom_fun in zip (noms_journaux,noms_function):

    l = getattr(st, nom_fun)(day)
    insert_nom_et_dates(nom_j, day, l)
    

https://www.liberation.fr/archives/2021/02/04


In [92]:
dir()

['Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Libération',
 'Le Monde

In [84]:
def test2(a):
    return a*2,a/2

In [86]:
g,gg=test2(5)

In [88]:
gg

2.5

In [81]:
test('a',l)

TypeError: can't multiply sequence by non-int of type 'list'

In [138]:
def scrap_libe(day):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'}
    liste_titre = []
    day_str = day.strftime('%Y/%m/%d')


    #url = "http://q.liberation.fr/archives/"+day_str+"/?page="+str(numero_page)
    url = "https://www.liberation.fr/archives/"+day_str
    req = Request(url , headers=headers)
    page = soup( urlopen(req).read() , "lxml")
    titres = page.find('div',{ 'class':"custom-card-list"}).findAll('h2',{ 'class':"font_tertiary font_xs font_normal"})
    for titre in titres:
        liste_titre.append(titre.getText().encode('utf-8').decode('utf-8').replace(u'\xa0', u' ').replace(u'\u200a', u' ').strip())
    return liste_titre

In [140]:
day

datetime.date(2021, 2, 3)

In [141]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta  
import time
import dateparser

scrap_libe(day)

['A Canal + et CNews, silence radio chez les journalistes',
 '«Voracisme» : «Pourquoi ce ne sont pas les Blancs qui font la plonge…»',
 'Internat pour les enfants : «Il est pensé pour les familles qui ont besoin d’aide»',
 'Jeu de milliards à trois bandes pour les médias de Lagardère',
 'La présence du virus dans les eaux usées en Ile-de-France a-t-elle baissé, comme l’affirme Véran ?',
 'La fabrique des «discriminations ressenties»',
 'Aux pieds d’Omar Sy dans «Lupin», les Air Jordan 1 suscitent un assaut de clics',
 'Macron sur le planning de vaccination, procès de l’affaire Karachi, les experts de l’OMS au laboratoire de Wuhan… l’actu de ce mercredi matin',
 'A Marseille, Payan étrille la gestion Gaudin',
 'La famille Berry se déchire autour d’accusations d’inceste',
 'En direct - Covid-19 : plus de 26 000 cas de contamination en 24 heures en France',
 '«Ma maman est bizarre» et tout va bien, merci',
 'Italie : Mario Draghi chargé de former un gouvernement',
 '«Il sera très difficil

In [106]:
def ru():
    global re
    re = re +2

    




In [115]:
def tt (nbb):
    return nbb+1

def tt2(nbb):
    return nbb*2

tt2(tt(5))

12

In [107]:
ru()

In [108]:
re

6

In [86]:
liste_th_chalie

['societe',
 'societe',
 'economie',
 'societe',
 'societe',
 'societe',
 'ecologie',
 'international',
 'societe',
 'economie',
 'societe',
 'societe',
 'societe',
 'societe',
 'ecologie',
 'economie',
 'societe',
 'societe',
 'societe',
 'sciences',
 'culture',
 'culture',
 'politique',
 'societe',
 'societe',
 'societe',
 'societe',
 'societe',
 'societe',
 'societe',
 'ecologie',
 'ecologie',
 'economie',
 'international',
 'societe',
 'societe',
 'societe',
 'societe',
 'ecologie',
 'culture',
 'societe',
 'societe',
 'societe',
 'ecologie',
 'sciences',
 'culture',
 'culture',
 'politique',
 'economie']

In [52]:
['Le Monde'] * 10

['Le Monde',
 'Le Monde',
 'Le Monde',
 'Le Monde',
 'Le Monde',
 'Le Monde',
 'Le Monde',
 'Le Monde',
 'Le Monde',
 'Le Monde']

In [37]:
day

datetime.date(2021, 2, 4)

In [8]:
st.recup_listes_charlie(pd.to_datetime('10-01-2021' , format="%d-%m-%Y"))

(['France, démocratie défaillante',
  'Guerre de 1870 : la mère de toutes les frustrations',
  'Élisabeth Guigou : une carrière en ne sachant rien',
  "La démocratie passera-t-elle l'hiver ?",
  "Anne Hidalgo : « Je n'ai qu'une boussole : l'universalisme »",
  'Aucune chance',
  'Mairie de Marseille : la cuisine électorale est dure à avaler',
  "L'État bien profond",
  'Vive la transparence politique',
  'Conseil des imams : le recteur de la Grande Mosquée de Paris, écœuré, lâche l’affaire',
  'La charité au programme de La République en marche',
  'Années Giscard : le retour de la droite rance et à particule',
  'Fichage des citoyens : la police est toute ouïe',
  "Les échappés du web : l'autorité, c'est tendance !",
  'Dernière causerie au coin du feu',
  "Aurélien Taché comme son nom l'indique",
  'Affaire du Sofitel : DSK prêt à nous livrer sa vérité',
  'Adieu VGE, un président « moderne »',
  '« En Auvergne, tout le monde ne pleure pas Giscard »',
  'Giscard, premier fossoyeur de

In [11]:
st.scrap_monde(pd.to_datetime('10-01-2021' , format="%d-%m-%Y"))

['Kazakhstan : victoire sans suspense du parti au pouvoir, avec plus de 70 % des voix',
 'A-genre, non-binaire... la neutralité de genre en question sur « Zone interdite »',
 'Le plasticien Claude Lévêque accusé de viols sur mineurs',
 'Algérie : le président Abdelmadjid Tebboune retourne se faire soigner en Allemagne',
 'La métropole lyonnaise étoffe son cabinet pour affronter la crise',
 'Couvre-feu avancé, neige en Espagne, crash en Indonésie : les cinq infos à retenir du week-end',
 'Au Kirghizistan, le populiste Sadyr Japarov élu président',
 'Michel Zaffran : « Eradiquer la poliomyélite est plus dur que prévu »',
 'De Madonna à Meghan Markle, le caraco caracole',
 '« Aux Etats-Unis, on ne cherche pas à ménager la sensibilité des vaccino-sceptiques »',
 '« Le rôle de tout responsable politique est de résister à la tentation de la polémique » : Jean Castex répond aux critiques',
 'Cyclo-cross : un quatrième titre de champion de France pour Clément Venturini',
 'Biathlon : une victo